In [ ]:
!pip install mysql-connector-python  

In [1]:
import mysql.connector  

connection = mysql.connector.connect(
    host="127.0.0.1",     
    user="root",         
    password="2011DongBaLiem@",
    database="preday50.1")
cursor = connection.cursor()

In [ ]:
import pandas as pd

In [2]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS customers(
    id INT PRIMARY KEY,
    name VARCHAR(100),
    city VARCHAR(100)
) ENGINE=InnoDB;
""")

cursor.execute("""
INSERT INTO customers VALUES
    (1, 'Alice', 'Hanoi'),
    (2, 'Bob', 'Ho Chi Minh'),
    (3, 'Charlie', 'Hanoi')
;
""") 
connection.commit()

cursor.execute("""
CREATE TABLE IF NOT EXISTS orders(
    id INT PRIMARY KEY,
    customer_id INT,
    order_date DATE,
    total INT,
    FOREIGN KEY(customer_id) REFERENCES customers(id)
)ENGINE=InnoDB;
""")

cursor.execute("""
INSERT INTO orders VALUES
    (1, 1, '2024-01-10', 500),
    (2, 1, '2024-03-01', 800),
    (3, 2, '2024-02-20', 1200),
    (4, 3, '2024-01-15', 200)
;
""")
connection.commit()

cursor.execute("""
CREATE TABLE IF NOT EXISTS products(
    id INT PRIMARY KEY,
    name VARCHAR(100),
    price INT
)ENGINE=InnoDB;
""")

cursor.execute("""
INSERT INTO products VALUES
    (1, 'Laptop', 1500),
    (2, 'Mouse', 50),
    (3, 'Keyboard', 100),
    (4, 'Monitor', 300)
;
""")
connection.commit()

cursor.execute("""
CREATE TABLE IF NOT EXISTS order_items(
    id INT PRIMARY KEY,
    order_id INT,
    product_id INT,
    quantity INT,
    FOREIGN KEY(product_id) REFERENCES products(id),
    FOREIGN KEY(order_id) REFERENCES orders(id)
)ENGINE=InnoDB;
""")

cursor.execute("""
INSERT INTO order_items VALUES
    (1, 1, 2, 2),
    (2, 1, 3, 1),
    (3, 2, 1, 1),
    (4, 2, 4, 1),
    (5, 3, 1, 1)
;
""")
connection.commit()

cursor.execute("""
CREATE TABLE IF NOT EXISTS employees(
    id INT PRIMARY KEY,
    name VARCHAR(100),
    department VARCHAR(100)
)ENGINE=InnoDB;
""")

cursor.execute("""
INSERT INTO employees VALUES
    (1, 'David', 'Sales'),
    (2, 'Emma', 'Support'),
    (3, 'Frank', 'Sales')
;
""")
connection.commit()

cursor.execute("""
CREATE TABLE IF NOT EXISTS order_assignments(
    id INT PRIMARY KEY,
    order_id INT,
    employee_id INT,
    FOREIGN KEY(order_id) REFERENCES orders(id),
    FOREIGN KEY(employee_id) REFERENCES employees(id)
)ENGINE=InnoDB;
""")

cursor.execute("""
INSERT INTO order_assignments VALUES
    (1, 1, 1),
    (2, 2, 3),
    (3, 3, 2)
;
""") 
connection.commit()

In [3]:
# Tạo trigger để ghi log vào bảng order_logs mỗi khi đơn hàng bị xoá khỏi bảng orders

query= """
CREATE TABLE IF NOT EXISTS order_logs (
    id INT AUTO_INCREMENT PRIMARY KEY,
    log_message TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
)
;"""
cursor.execute(query)

In [4]:
query="""
CREATE TRIGGER log_order_delete
AFTER DELETE ON orders 
FOR EACH ROW 
BEGIN 
    INSERT INTO order_logs(log_message)
    VALUES (CONCAT('Delete order ID:', OLD.id, '-total:', OLD.total));
END
;"""

cursor.execute(query)
connection.commit()

In [7]:
# TEST TRIGGER
query="""
DELETE FROM order_assignments
WHERE order_id=1;"""
cursor.execute(query)

In [8]:
query="""
DELETE FROM order_items
WHERE order_id=1;"""
cursor.execute(query)

In [9]:
query="""
DELETE FROM orders
WHERE id=1;"""
cursor.execute(query)

In [10]:
query=""" SELECT * FROM order_logs"""
cursor.execute(query)
result = cursor.fetchall()
for row in result:
    print(row)

(1, 'Delete order ID:1-total:500', datetime.datetime(2025, 4, 14, 14, 56, 36))


In [11]:
# Tạo trigger trên bảng order_items để tự động cập nhật tổng tiền của đơn hàng trong bảng orders khi thêm dòng sản phẩm mới

query="""
CREATE TABLE update_product_logs(
    id INT AUTO_INCREMENT PRIMARY KEY,
    log_message TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
)
;"""
cursor.execute(query)

In [12]:
query= """
CREATE TRIGGER update_order_total
AFTER INSERT ON order_items
FOR EACH ROW
BEGIN
    UPDATE orders
    SET total = total + NEW.quantity * (SELECT price FROM products WHERE id= NEW.product_id)
    WHERE id = NEW.order_id;

    INSERT INTO update_product_logs(log_message) 
    VALUES (CONCAT('NEW order_id:', NEW.order_id, '-product_id:', NEW.product_id, '-quantity:', NEW.quantity)) ;
END
;""" 

cursor.execute(query)
connection.commit()

In [13]:
# TEST TRIGGER
query="""
INSERT INTO products VALUES
    (5,'Main', 200)
;"""
cursor.execute(query)
connection.commit()

In [14]:
query= """
INSERT INTO order_items VALUES
    (6,2,5,1)
;"""
cursor.execute(query)
connection.commit()

ProgrammingError: 1054 (42S22): Unknown column 'order_id' in 'where clause'